In [ ]:
import pandas as pd
import ast

There are many patient notes without annotations. Making good use of them may be very important in this competition.

In [ ]:
features = pd.read_csv('/kaggle/input/nbme-score-clinical-patient-notes/features.csv')

In [ ]:
train = pd.read_csv('/kaggle/input/nbme-score-clinical-patient-notes/train.csv')

In [ ]:
pn = pd.read_csv('/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv')

In [ ]:
pn_dict = {}
for idx, row in pn.iterrows():
    pn_dict[row['pn_num']] = row['pn_history']

In [ ]:
new_annotation = []
for case_id in features['case_num'].unique():
    
    all_pn_id = set(pn[pn['case_num']==case_id]['pn_num'].tolist())
    
    for feature_id in features[features['case_num']==case_id]['feature_num'].unique():
        # get all the pn_num that have already been annotated
        annotated_pn = set(train[train['feature_num']==feature_id]['pn_num'].tolist())
        # get all the pn_num that have NOT been annotated
        pn_to_annotate = all_pn_id-annotated_pn
        
        # get all current annotations
        # we will use them to find more annotations
        annotations = train[train['feature_num']==feature_id]['annotation'].tolist()
        annotation_texts = set()
        for a in annotations:
            anns = eval(a)
            for at in anns:
                annotation_texts.add(at)
                
        # annotate       
        for pn_id in pn_to_annotate:
            new_annotation_pn, new_location_pn = [], []
            pn_text = pn_dict[pn_id]
            for at in annotation_texts:
                start = pn_text.find(at)
                if start>=0:
                    new_annotation_pn.append(at)
                    new_location_pn.append(f'{start} {start+len(at)}')
            if len(new_annotation_pn)>0:
                new_annotation.append((
                    f'{pn_id:04d}_{feature_id:03d}',
                    case_id,
                    pn_id,
                    feature_id,
                    new_annotation_pn,
                    new_location_pn
                ))
        break
    break
    # break to get sample results quickly

In [ ]:
new_annotation[:10]

In [ ]:
len(new_annotation)

Great! Seems we get many new annotations!

In [ ]:
# check the text
train[(train['feature_num']==0)&(train['case_num']==0)]['annotation'].unique()

In [ ]:
# make sure new annotation is new
assert '0000_000' not in train['id'].unique()
assert '0025_000' not in train['id'].unique()

In [ ]:
train[train['pn_num']==0]

In [ ]:
# check one newly annotated patient note
# we should see "father had MI" in the text
print(pn_dict[0])

Hope this helps :)

As a Z by HP Global Data Science Ambassador, Yuanhao's content is sponsored and he was provided with HP products.